# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,oshnaviyeh,37.0397,45.0983,284.59,68,33,2.63,IR,1697668118
1,1,ilulissat,69.2167,-51.1000,263.16,67,75,1.54,GL,1697668118
2,2,new norfolk,-42.7826,147.0587,288.55,50,98,1.05,AU,1697668118
3,3,rongelap,11.1635,166.8941,300.14,80,100,6.70,MH,1697668118
4,4,adamstown,-25.0660,-130.1015,295.89,78,65,3.30,PN,1697667991


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    hover_cols=['City', 'Country', 'Humidity'],
    colorbar=True,
    color='Humidity',
    title='City Humidity Map'
)

# Display the map
map_plot.opts(width=800, height=600)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df[['Max Temp', 'Wind Speed', 'Cloudiness']].describe()

,Max Temp,Wind Speed,Cloudiness
count,543.000000,543.000000,543.000000
mean,290.347090,3.640387,56.703499
std,10.439464,2.838483,39.141178
min,255.350000,0.000000,0.000000
25%,283.060000,1.540000,20.000000
50%,292.260000,3.090000,65.000000
75%,298.795000,4.695000,99.000000
max,313.100000,29.500000,100.000000


In [5]:

# Narrow down cities that fit criteria and drop any results with null values
lower_temp_limit = 21 + 273.15  # 21 degrees Celsius in Kelvin
upper_temp_limit = 27 + 273.15  # 27 degrees Celsius in Kelvin

ideal_temperature_range = (city_data_df["Max Temp"] >= lower_temp_limit) & (city_data_df["Max Temp"] <= upper_temp_limit)
low_wind_speed = city_data_df["Wind Speed"] < 4.5
clear_sky = city_data_df["Cloudiness"] == 0

ideal_cities = city_data_df[ideal_temperature_range & low_wind_speed & clear_sky]


# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,rakops,-21.0167,24.3333,294.86,34,0,3.73,BW,1697668119
53,53,nova sintra,14.8667,-24.7167,298.08,70,0,3.77,CV,1697668121
145,145,hutchinson,38.0608,-97.9298,294.68,35,0,2.06,US,1697668059
212,212,bishunpur urf maharajganj,26.2519,83.1186,294.25,65,0,2.50,IN,1697668130
230,230,broken hill,-31.9500,141.4333,294.35,27,0,2.43,AU,1697667859
324,324,tazacorte,28.6290,-17.9293,296.69,78,0,2.57,ES,1697668137
386,386,remire-montjoly,4.9167,-52.2667,298.17,94,0,1.03,GF,1697668140
420,420,robbah,33.2797,6.9098,295.95,19,0,2.53,DZ,1697668143
433,433,waddan,29.1614,16.1390,297.77,20,0,3.45,LY,1697668144
441,441,callaway,30.1530,-85.5699,296.18,62,0,3.09,US,1697668144


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,rakops,BW,-21.0167,24.3333,34,
53,nova sintra,CV,14.8667,-24.7167,70,
145,hutchinson,US,38.0608,-97.9298,35,
212,bishunpur urf maharajganj,IN,26.2519,83.1186,65,
230,broken hill,AU,-31.9500,141.4333,27,
324,tazacorte,ES,28.6290,-17.9293,78,
386,remire-montjoly,GF,4.9167,-52.2667,94,
420,robbah,DZ,33.2797,6.9098,19,
433,waddan,LY,29.1614,16.1390,20,
441,callaway,US,30.1530,-85.5699,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "limit": 1,
    "categories": "accommodation",
    "format": "json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


rakops - nearest hotel: Rakops River Lodge
nova sintra - nearest hotel: BAR DOS AMIGOS CAMPO - MOTEL and BAR RESTAURANT
hutchinson - nearest hotel: Econo Lodge
bishunpur urf maharajganj - nearest hotel: No hotel found
broken hill - nearest hotel: Sturt Motel
tazacorte - nearest hotel: Finca Marina BuenVivir
remire-montjoly - nearest hotel: Motel du Lac
robbah - nearest hotel: المجمع الشباني المجاهد بالصالة المولدي
waddan - nearest hotel: بيت جدي
callaway - nearest hotel: Holiday Inn Express & Suites Panama City-Tyndall
rahimah - nearest hotel: No hotel found
matoury - nearest hotel: Casa Blanca


,City,Country,Lat,Lng,Humidity,Hotel Name
16,rakops,BW,-21.0167,24.3333,34,Rakops River Lodge
53,nova sintra,CV,14.8667,-24.7167,70,BAR DOS AMIGOS CAMPO - MOTEL and BAR RESTAURANT
145,hutchinson,US,38.0608,-97.9298,35,Econo Lodge
212,bishunpur urf maharajganj,IN,26.2519,83.1186,65,No hotel found
230,broken hill,AU,-31.9500,141.4333,27,Sturt Motel
324,tazacorte,ES,28.6290,-17.9293,78,Finca Marina BuenVivir
386,remire-montjoly,GF,4.9167,-52.2667,94,Motel du Lac
420,robbah,DZ,33.2797,6.9098,19,المجمع الشباني المجاهد بالصالة المولدي
433,waddan,LY,29.1614,16.1390,20,بيت جدي
441,callaway,US,30.1530,-85.5699,62,Holiday Inn Express & Suites Panama City-Tyndall


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hover_columns = ['Hotel Name', 'Country']

map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    hover_cols=hover_columns,
    color='Humidity',
    size='Humidity',
    width=600,
    height=400,
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)